# 0. Web APIを用いたマンガ雑誌データの取得

[文化庁メディア芸術データベース　マンガ分野　WebAPI](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)を用いて，分析に必要なデータを入手します．なお，python3を使ったweb APIの利用については，[Python3でjsonを返却するwebAPIにアクセスして結果を出力するまで](http://qiita.com/sakura1116/items/4a11a9f7db9f535397fa)を参考にさせて頂きました．

## 環境構築

```bash
conda env create -f env.yml
```

## 準備

In [1]:
import json
import urllib.request

## 雑誌巻号検索結果の取得

週刊少年ジャンプの雑誌巻号情報を検索して，結果を変数`magazines`に保存します．ここで取得したユニークIDは，次節の「雑誌巻号情報の取得」に必要になります．

なお，2017年3月31日より文化庁メディア芸術データベースのURLが変更されたため，[WebAPI仕様](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)に記載のリクエストURL（https://mediaarts-db.jp/mg/api/v1/results_magazines　）ではなく，新しいURL（https://mediaarts-db.bunka.go.jp/mg/api/v1/results_magazines　）を使用する必要があることにご注意ください．

In [10]:
url = 'https://mediaarts-db.bunka.go.jp/mg/api/v1/results_magazines' + \
    '?id=JUMPrgl&page='

リクエストパラメータ`id`で「ユニークID」「雑誌巻号ID」あるいは「雑誌コード」を，`page`で検索結果の取得ページ番号（1ページあたり100件，デフォルトは1）を指定することができます．ここで，週刊少年ジャンプは「雑誌巻号ID」に`JUMPrgl`を含むため，`id=JUMPrgl`を指定します．また，週刊少年ジャンプの検索結果は合計24ページ（2320件）あるため，`page`に1から24を順次指定する必要があります．
詳細は[WebAPI仕様](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)をご参照ください．

In [11]:
magazines = []
for i in range(1, 25):
    response = urllib.request.urlopen(url + str(i))
    content = json.loads(response.read().decode('utf8'))
    magazines.extend(content['results'])

In [12]:
len(magazines)

2320

In [13]:
magazines[0]

{'date_indication': '1969/11/03',
 'id': 323270,
 'magazine_id': 'JUMPrgl19691103',
 'name': '週刊少年ジャンプ',
 'number_assist': '',
 'number_indication': '20',
 'owner_kawasaki': 0,
 'owner_kikuyo': 0,
 'owner_kyoto': 1,
 'owner_meiji': 0,
 'owner_ndl': 0,
 'owner_osaka': 0,
 'publisher': '集英社',
 'vol_num_totalval': '2巻24号(通巻35)'}

In [14]:
magazines[-1]

{'date_indication': '2016/12/12',
 'id': 475979,
 'magazine_id': 'JUMPrgl20161212',
 'name': '週刊少年ジャンプ',
 'number_assist': '',
 'number_indication': '52',
 'owner_kawasaki': 0,
 'owner_kikuyo': 0,
 'owner_kyoto': 0,
 'owner_meiji': 0,
 'owner_ndl': 1,
 'owner_osaka': 0,
 'publisher': '',
 'vol_num_totalval': '49巻48号(通巻2389)'}

## 雑誌巻号情報の取得

上記の`magazines`を用い，雑誌巻号情報を取得し，変数`comics`に保存します．

上記と同様に，2017年3月31日より文化庁メディア芸術データベースのURLが変更されたため，[WebAPI仕様](https://mediaarts-db.bunka.go.jp/webapi_proto_documents.pdf)に記載のリクエストURL（https://mediaarts-db.jp/mg/api/v1/magazine　）ではなく，新しいURL（https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine　）を使用する必要があることにご注意ください．

In [28]:
url = 'https://mediaarts-db.bunka.go.jp/mg/api/v1/magazine?id='

In [29]:
response = urllib.request.urlopen(url + str(magazines[0]['id']))
content = json.loads(response.read().decode('utf8'))
content['contents']

[{'author': '',
  'category': '表紙',
  'form': '',
  'note': '4色カラー',
  'start_page': 1.0,
  'subtitle': '[モサ\u3000／\u3000どうどう野郎\u3000／\u3000男一匹ガキ大将]',
  'work': '[表紙]',
  'work_pages': 1.0},
 {'author': '永井豪とダイナミックプロ',
  'category': 'マンガ作品',
  'form': '',
  'note': '4色カラー',
  'start_page': 7.0,
  'subtitle': '赤い嵐の巻',
  'work': 'ハレンチ学園',
  'work_pages': 31.0},
 {'author': 'ちばてつや',
  'category': 'マンガ作品',
  'form': '',
  'note': '',
  'start_page': 39.0,
  'subtitle': '盗まれた金の巻',
  'work': 'モサ',
  'work_pages': 15.0},
 {'author': '榎本有也',
  'category': 'マンガ作品',
  'form': '',
  'note': '',
  'start_page': 54.0,
  'subtitle': 'ゆうかい魔の巻',
  'work': 'まんがコント55号',
  'work_pages': 2.0},
 {'author': '川崎のぼるとカワサキプロ',
  'category': 'マンガ作品',
  'form': '',
  'note': '',
  'start_page': 56.0,
  'subtitle': '転校生の巻',
  'work': 'どうどう野郎',
  'work_pages': 15.0},
 {'author': '本宮ひろ志',
  'category': 'マンガ作品',
  'form': '',
  'note': '',
  'start_page': 71.0,
  'subtitle': '万吉仁王立ちの巻',
  'work': '男一匹ガキ大将',
  'work_p

とりあえずこれで，必要な情報は取り出せそう．あとは，`wj.json`と整合するように加工するのみ．
一括書き込みではなく，逐次書き込みができるようにしたい．jsonで書き込まず，テキストとしてベタ打ちするのもひとつの手かもしれない．まずは，json.dumpで逐次書き込みができるか，試してみること．ダメだったらテキストとして書き込む．